In [14]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
relation_feat_syn = None
relation_feat_ant = None
relation_feat_hype = None
relation_feat_hypo = None
relation_feat_same_hype = None
def PrintRelationLst():
    print('relation-syn:',relation_feat_syn)
    print('relation-ant:',relation_feat_ant)
    print('relation-hype:',relation_feat_hype)
    print('relation-hypo:',relation_feat_hypo)
    print('relation-same_hype:',relation_feat_same_hype)

## Synonymy & Antonymy

In [2]:
# WORD1 = 'good'
# WORD2 = 'beneficial'
# WORD1 = 'wet'
# WORD2 = 'dry'
WORD1 = 'wet'
WORD2 = 'dry'
syn_flg = False
WORD1_syn_lst = wn.synsets(WORD1)
WORD2_syn_lst =wn.synsets(WORD2)
# synonymy
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        if word1_syn == word2_syn:
            relation_feat_syn = 1
    
# antonymy
syn_flg = True
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        if word1_syn.lemmas()[0].antonyms():
#             print('word1:',word1_syn.lemmas()[0])
#             print('word2:',word2_syn.lemmas()[0])
#             print('============================')
            if word1_syn.lemmas()[0].antonyms()[0] == word2_syn.lemmas()[0]:
                relation_feat_ant = 1
if not relation_feat_syn:
    relation_feat_syn = 0
if not relation_feat_ant:
    relation_feat_ant = 0
PrintRelationLst()

relation-syn: 0
relation-ant: 1
relation-hype: None
relation-hypo: None
relation-same_hype: None


## Hypernymy & Hpyponymy & Same-Hypernym

In [3]:
# here due to its external knowledge definition, we will simple consider Hypernymy and Hyponymy as the same value
# =========== def ===========
# takes value 1 - n/8 if one word is a (direct or indirect) hypernym of the other wors in WordNet. 
# Where n is the number of edges between the two words in hierarchies, and 0 otherwise.
# Eg. [dog, canid] = 0.875
# Eg. [wolf, canid] = 0.875
# Eg. [dog, carnivore] = 0.75
# Eg. [canid, dog] = 0
# ======== End def ==========

## first find their most similarity def
print('word1 = ',WORD1)
print('word2 = ',WORD2)
print('================')
similarity = -np.float('inf')
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        sim_val = wn.wup_similarity(word1_syn, word2_syn)
        if sim_val: # if they exist similaroty value
            if sim_val > similarity:
                best_val = sim_val
                best_syn1 = word1_syn
                best_syn2 = word2_syn
                similarity = sim_val
# print(best_syn1)
# print(best_syn2)
        
## base on the def, to figure out their hyper-path
best_syn1_path = best_syn1.hypernym_paths()
for idx, path in enumerate(best_syn1_path):
    for i, hyper in enumerate(path):
        if hyper == best_syn2:
            edgN = len(path)-i-1
            relation_feat_hype = 1-edgN/8
            break
    if relation_feat_hype:
        break
    
best_syn2_path = best_syn2.hypernym_paths()
for idx, path in enumerate(best_syn2_path):
    for i, hyper in enumerate(path):
        if hyper == best_syn1:
            edgN = len(path)-i-1
            relation_feat_hypo = 1-edgN/8
            break
    if relation_feat_hypo:
        break

if relation_feat_hype:
    relation_feat_hypo = relation_feat_hype
elif relation_feat_hypo:
    relation_feat_hype = relation_feat_hypo
else:
    relation_feat_hype = 0
    relation_feat_hypo = 0
    
## Same-Hypernym
if relation_feat_syn == 0:
    # to see if the two words have the same hypernym
    for syn1_idx, syn1_path in enumerate(best_syn1_path):
        for syn2_idx, syn2_path in enumerate(best_syn2_path):
            for syn1_hyper in syn1_path:
                for syn2_hyper in syn2_path:
                    if syn1_hyper == syn2_hyper:
#                         print(syn1_hyper)
#                         print(syn2_hyper)
                        relation_feat_same_hype = 1
if not relation_feat_same_hype:
    relation_feat_same_hype = 0
# PrintRelationLst() 

word1 =  wet
word2 =  dry


In [4]:
PrintRelationLst()

relation-syn: 0
relation-ant: 1
relation-hype: 0
relation-hypo: 0
relation-same_hype: 0


In [5]:
## External knowledge
'''
rij = [syn, ant, hype, hypo, same-hype]
λ1(rij ), where λ is a hyper-parameter tuned on the development set 
and 1 is the indication function.
1(rij ) = 1 if rij is not zero vector ;
1(rij ) = 0 if rij is zero vector .
'''
r = [relation_feat_syn, relation_feat_ant, relation_feat_hype, relation_feat_hypo, relation_feat_same_hype] # r_ij
if all(ele == 0 for ele in r):
    lr = 0 # lr_ij
else:
    lr = 1 # lr_ij

## from SNLI-dataset to build a relation-feature dictionary

In [33]:
## load data
import json
import random    
import string

LABEL_MAP = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2,
    "hidden": 0
}

def load_nli_data(path, snli=False):
    """
    Load MultiNLI or SNLI data.
    If the "snli" parameter is set to True, a genre label of snli will be assigned to the data. 
    """
    data = []
    with open(path) as f:
        for line in f:
            loaded_example = json.loads(line)
            if loaded_example["gold_label"] not in LABEL_MAP:
                continue
            loaded_example["label"] = LABEL_MAP[loaded_example["gold_label"]]
            if snli:
                loaded_example["genre"] = "snli"
            data.append(loaded_example)
        random.seed(1)
        random.shuffle(data)
    return data

test_snli = load_nli_data("./data/snli_1.0/snli_1.0_test.jsonl", snli=True)

In [48]:
def relation_feat(WORD1, WORD2):
    # init
    relation_feat_syn = None
    relation_feat_ant = None
    relation_feat_hype = None
    relation_feat_hypo = None
    relation_feat_same_hype = None
    
    syn_flg = False
    WORD1_syn_lst = wn.synsets(WORD1)
    WORD2_syn_lst =wn.synsets(WORD2)
    if not WORD1_syn_lst or not WORD2_syn_lst:
        return [0,0,0,0,0]
    
    # synonymy
    for word1_i, word1_syn in enumerate(WORD1_syn_lst):
        for word2_i, word2_syn in enumerate(WORD2_syn_lst):
            if word1_syn == word2_syn:
                relation_feat_syn = 1

    # antonymy
    syn_flg = True
    for word1_i, word1_syn in enumerate(WORD1_syn_lst):
        for word2_i, word2_syn in enumerate(WORD2_syn_lst):
            if word1_syn.lemmas()[0].antonyms():
                if word1_syn.lemmas()[0].antonyms()[0] == word2_syn.lemmas()[0]:
                    relation_feat_ant = 1
    if not relation_feat_syn:
        relation_feat_syn = 0
    if not relation_feat_ant:
        relation_feat_ant = 0
        
    # hypernymy&hyponymy
    ## first find their most similarity def
    similarity = -np.float('inf')
    sim_flag = False
    for word1_i, word1_syn in enumerate(WORD1_syn_lst):
        for word2_i, word2_syn in enumerate(WORD2_syn_lst):
            sim_val = wn.wup_similarity(word1_syn, word2_syn)
            if sim_val: # if they exist similaroty value
                sim_flag = True
                if sim_val > similarity:
                    best_val = sim_val
                    best_syn1 = word1_syn
                    best_syn2 = word2_syn
                    similarity = sim_val
    if not sim_flag: # in case the 2 words don't exist any similarity definision synset.
        return [0,0,0,0,0]
    
    ## base on the def, to figure out their hyper-path
    best_syn1_path = best_syn1.hypernym_paths()
    for idx, path in enumerate(best_syn1_path):
        for i, hyper in enumerate(path):
            if hyper == best_syn2:
                edgN = len(path)-i-1
                relation_feat_hype = 1-edgN/8
                break
        if relation_feat_hype:
            break

    best_syn2_path = best_syn2.hypernym_paths()
    for idx, path in enumerate(best_syn2_path):
        for i, hyper in enumerate(path):
            if hyper == best_syn1:
                edgN = len(path)-i-1
                relation_feat_hypo = 1-edgN/8
                break
        if relation_feat_hypo:
            break

    if relation_feat_hype:
        relation_feat_hypo = relation_feat_hype
    elif relation_feat_hypo:
        relation_feat_hype = relation_feat_hypo
    else:
        relation_feat_hype = 0
        relation_feat_hypo = 0

    ## Same-Hypernym
    if relation_feat_syn == 0:
        # to see if the two words have the same hypernym
        for syn1_idx, syn1_path in enumerate(best_syn1_path):
            for syn2_idx, syn2_path in enumerate(best_syn2_path):
                for syn1_hyper in syn1_path:
                    for syn2_hyper in syn2_path:
                        if syn1_hyper == syn2_hyper:
    #                         print(syn1_hyper)
    #                         print(syn2_hyper)
                            relation_feat_same_hype = 1
    if not relation_feat_same_hype:
        relation_feat_same_hype = 0
        
    return [relation_feat_syn, relation_feat_ant, relation_feat_hype, relation_feat_hypo, relation_feat_same_hype]

# remove all punctuations (except while space)
def preprocess_word(str_word):
    signtext = string.punctuation 
    signrepl = '@'*len(signtext)
    signtable = str_word.maketrans(signtext,signrepl) 
    return str_word.translate(signtable).replace('@','')

Dic_external_know = {}
for data_i, data_ in enumerate(test_snli):
    print('Process to {}th sentences.'.format(data_i))
    sent_1 = data_['sentence1']
    sent_2 = data_['sentence2'] 
    sent_1 = preprocess_word(sent_1) # remove all punctuations
    sent_2 = preprocess_word(sent_2) # remove all punctuations
    for word1 in sent_1.split():
        for word2 in sent_2.split():
            R_lst = relation_feat(word1, word2) # five relation feature value: [ayn, ant, hype, hypo, same-hype]
            if all(ele == 0 for ele in R_lst):
                l_R = 0 # lr_ij
            else:
                l_R = 1 # lr_ij
            Dic_external_know[(word1,word2)] = l_R # key: (word1, word2); value: L(R_ij) [should be 1 or 0]
    




Process to 0th sentences.
Process to 1th sentences.
Process to 2th sentences.
Process to 3th sentences.
Process to 4th sentences.
Process to 5th sentences.
Process to 6th sentences.
Process to 7th sentences.
Process to 8th sentences.
Process to 9th sentences.
Process to 10th sentences.
Process to 11th sentences.
Process to 12th sentences.
Process to 13th sentences.
Process to 14th sentences.
Process to 15th sentences.
Process to 16th sentences.
Process to 17th sentences.
Process to 18th sentences.
Process to 19th sentences.
Process to 20th sentences.
Process to 21th sentences.
Process to 22th sentences.
Process to 23th sentences.
Process to 24th sentences.
Process to 25th sentences.
Process to 26th sentences.
Process to 27th sentences.
Process to 28th sentences.
Process to 29th sentences.
Process to 30th sentences.
Process to 31th sentences.
Process to 32th sentences.
Process to 33th sentences.
Process to 34th sentences.
Process to 35th sentences.
Process to 36th sentences.
Process to 

Process to 298th sentences.
Process to 299th sentences.
Process to 300th sentences.
Process to 301th sentences.
Process to 302th sentences.
Process to 303th sentences.
Process to 304th sentences.
Process to 305th sentences.
Process to 306th sentences.
Process to 307th sentences.
Process to 308th sentences.
Process to 309th sentences.
Process to 310th sentences.
Process to 311th sentences.
Process to 312th sentences.
Process to 313th sentences.
Process to 314th sentences.
Process to 315th sentences.
Process to 316th sentences.
Process to 317th sentences.
Process to 318th sentences.
Process to 319th sentences.
Process to 320th sentences.
Process to 321th sentences.
Process to 322th sentences.
Process to 323th sentences.
Process to 324th sentences.
Process to 325th sentences.
Process to 326th sentences.
Process to 327th sentences.
Process to 328th sentences.
Process to 329th sentences.
Process to 330th sentences.
Process to 331th sentences.
Process to 332th sentences.
Process to 333th sen

Process to 591th sentences.
Process to 592th sentences.
Process to 593th sentences.
Process to 594th sentences.
Process to 595th sentences.
Process to 596th sentences.
Process to 597th sentences.
Process to 598th sentences.
Process to 599th sentences.
Process to 600th sentences.
Process to 601th sentences.
Process to 602th sentences.
Process to 603th sentences.
Process to 604th sentences.
Process to 605th sentences.
Process to 606th sentences.
Process to 607th sentences.
Process to 608th sentences.
Process to 609th sentences.
Process to 610th sentences.
Process to 611th sentences.
Process to 612th sentences.
Process to 613th sentences.
Process to 614th sentences.
Process to 615th sentences.
Process to 616th sentences.
Process to 617th sentences.
Process to 618th sentences.
Process to 619th sentences.
Process to 620th sentences.
Process to 621th sentences.
Process to 622th sentences.
Process to 623th sentences.
Process to 624th sentences.
Process to 625th sentences.
Process to 626th sen

Process to 884th sentences.
Process to 885th sentences.
Process to 886th sentences.
Process to 887th sentences.
Process to 888th sentences.
Process to 889th sentences.
Process to 890th sentences.
Process to 891th sentences.
Process to 892th sentences.
Process to 893th sentences.
Process to 894th sentences.
Process to 895th sentences.
Process to 896th sentences.
Process to 897th sentences.
Process to 898th sentences.
Process to 899th sentences.
Process to 900th sentences.
Process to 901th sentences.
Process to 902th sentences.
Process to 903th sentences.
Process to 904th sentences.
Process to 905th sentences.
Process to 906th sentences.
Process to 907th sentences.
Process to 908th sentences.
Process to 909th sentences.
Process to 910th sentences.
Process to 911th sentences.
Process to 912th sentences.
Process to 913th sentences.
Process to 914th sentences.
Process to 915th sentences.
Process to 916th sentences.
Process to 917th sentences.
Process to 918th sentences.
Process to 919th sen

Process to 1171th sentences.
Process to 1172th sentences.
Process to 1173th sentences.
Process to 1174th sentences.
Process to 1175th sentences.
Process to 1176th sentences.
Process to 1177th sentences.
Process to 1178th sentences.
Process to 1179th sentences.
Process to 1180th sentences.
Process to 1181th sentences.
Process to 1182th sentences.
Process to 1183th sentences.
Process to 1184th sentences.
Process to 1185th sentences.
Process to 1186th sentences.
Process to 1187th sentences.
Process to 1188th sentences.
Process to 1189th sentences.
Process to 1190th sentences.
Process to 1191th sentences.
Process to 1192th sentences.
Process to 1193th sentences.
Process to 1194th sentences.
Process to 1195th sentences.
Process to 1196th sentences.
Process to 1197th sentences.
Process to 1198th sentences.
Process to 1199th sentences.
Process to 1200th sentences.
Process to 1201th sentences.
Process to 1202th sentences.
Process to 1203th sentences.
Process to 1204th sentences.
Process to 120

...

Process to 8530th sentences.
Process to 8531th sentences.
Process to 8532th sentences.
Process to 8533th sentences.
Process to 8534th sentences.
Process to 8535th sentences.
Process to 8536th sentences.
Process to 8537th sentences.
Process to 8538th sentences.
Process to 8539th sentences.
Process to 8540th sentences.
Process to 8541th sentences.
Process to 8542th sentences.
Process to 8543th sentences.
Process to 8544th sentences.
Process to 8545th sentences.
Process to 8546th sentences.
Process to 8547th sentences.
Process to 8548th sentences.
Process to 8549th sentences.
Process to 8550th sentences.
Process to 8551th sentences.
Process to 8552th sentences.
Process to 8553th sentences.
Process to 8554th sentences.
Process to 8555th sentences.
Process to 8556th sentences.
Process to 8557th sentences.
Process to 8558th sentences.
Process to 8559th sentences.
Process to 8560th sentences.
Process to 8561th sentences.
Process to 8562th sentences.
Process to 8563th sentences.
Process to 856

Process to 8813th sentences.
Process to 8814th sentences.
Process to 8815th sentences.
Process to 8816th sentences.
Process to 8817th sentences.
Process to 8818th sentences.
Process to 8819th sentences.
Process to 8820th sentences.
Process to 8821th sentences.
Process to 8822th sentences.
Process to 8823th sentences.
Process to 8824th sentences.
Process to 8825th sentences.
Process to 8826th sentences.
Process to 8827th sentences.
Process to 8828th sentences.
Process to 8829th sentences.
Process to 8830th sentences.
Process to 8831th sentences.
Process to 8832th sentences.
Process to 8833th sentences.
Process to 8834th sentences.
Process to 8835th sentences.
Process to 8836th sentences.
Process to 8837th sentences.
Process to 8838th sentences.
Process to 8839th sentences.
Process to 8840th sentences.
Process to 8841th sentences.
Process to 8842th sentences.
Process to 8843th sentences.
Process to 8844th sentences.
Process to 8845th sentences.
Process to 8846th sentences.
Process to 884

Process to 9096th sentences.
Process to 9097th sentences.
Process to 9098th sentences.
Process to 9099th sentences.
Process to 9100th sentences.
Process to 9101th sentences.
Process to 9102th sentences.
Process to 9103th sentences.
Process to 9104th sentences.
Process to 9105th sentences.
Process to 9106th sentences.
Process to 9107th sentences.
Process to 9108th sentences.
Process to 9109th sentences.
Process to 9110th sentences.
Process to 9111th sentences.
Process to 9112th sentences.
Process to 9113th sentences.
Process to 9114th sentences.
Process to 9115th sentences.
Process to 9116th sentences.
Process to 9117th sentences.
Process to 9118th sentences.
Process to 9119th sentences.
Process to 9120th sentences.
Process to 9121th sentences.
Process to 9122th sentences.
Process to 9123th sentences.
Process to 9124th sentences.
Process to 9125th sentences.
Process to 9126th sentences.
Process to 9127th sentences.
Process to 9128th sentences.
Process to 9129th sentences.
Process to 913

Process to 9379th sentences.
Process to 9380th sentences.
Process to 9381th sentences.
Process to 9382th sentences.
Process to 9383th sentences.
Process to 9384th sentences.
Process to 9385th sentences.
Process to 9386th sentences.
Process to 9387th sentences.
Process to 9388th sentences.
Process to 9389th sentences.
Process to 9390th sentences.
Process to 9391th sentences.
Process to 9392th sentences.
Process to 9393th sentences.
Process to 9394th sentences.
Process to 9395th sentences.
Process to 9396th sentences.
Process to 9397th sentences.
Process to 9398th sentences.
Process to 9399th sentences.
Process to 9400th sentences.
Process to 9401th sentences.
Process to 9402th sentences.
Process to 9403th sentences.
Process to 9404th sentences.
Process to 9405th sentences.
Process to 9406th sentences.
Process to 9407th sentences.
Process to 9408th sentences.
Process to 9409th sentences.
Process to 9410th sentences.
Process to 9411th sentences.
Process to 9412th sentences.
Process to 941

Process to 9662th sentences.
Process to 9663th sentences.
Process to 9664th sentences.
Process to 9665th sentences.
Process to 9666th sentences.
Process to 9667th sentences.
Process to 9668th sentences.
Process to 9669th sentences.
Process to 9670th sentences.
Process to 9671th sentences.
Process to 9672th sentences.
Process to 9673th sentences.
Process to 9674th sentences.
Process to 9675th sentences.
Process to 9676th sentences.
Process to 9677th sentences.
Process to 9678th sentences.
Process to 9679th sentences.
Process to 9680th sentences.
Process to 9681th sentences.
Process to 9682th sentences.
Process to 9683th sentences.
Process to 9684th sentences.
Process to 9685th sentences.
Process to 9686th sentences.
Process to 9687th sentences.
Process to 9688th sentences.
Process to 9689th sentences.
Process to 9690th sentences.
Process to 9691th sentences.
Process to 9692th sentences.
Process to 9693th sentences.
Process to 9694th sentences.
Process to 9695th sentences.
Process to 969

In [49]:
import pickle
output = open('snli_test_relationship_feat.pkl', 'wb')
pickle.dump(Dic_external_know, output)
output.close()

In [50]:
Dic_external_know

{('Runners', 'are'): 1,
 ('group', 'score'): 1,
 ('suckers', 'running'): 1,
 ('People', 'produce'): 0,
 ('play', 'field'): 1,
 ('skateboarder', 'crowd'): 1,
 ('in', 's'): 1,
 ('trunk', 'his'): 0,
 ('bib', 'firetruck'): 0,
 ('girls', 'under'): 0,
 ('riding', 'dirt'): 1,
 ('festival', 'of'): 0,
 ('solo', 'and'): 0,
 ('flowers', 'violets'): 1,
 ('walking', 'traffic'): 1,
 ('young', 'post'): 1,
 ('luck', 'a'): 1,
 ('Team', 'jogging'): 1,
 ('costume', 'livingrooms'): 0,
 ('between', 'vehicles'): 0,
 ('Centre', 'just'): 0,
 ('scarf', 'all'): 0,
 ('job', 'forest'): 1,
 ('Two', 'jumping'): 1,
 ('house', 'standing'): 1,
 ('has', 'Armenian'): 0,
 ('goes', 'blue'): 1,
 ('bicyclist', 'a'): 1,
 ('tiny', 'humans'): 0,
 ('Four', 'gym'): 1,
 ('is', 'guys'): 0,
 ('food', 'Chinese'): 1,
 ('Walmart', 'hill'): 0,
 ('button', 'shits'): 1,
 ('The', 'snow'): 0,
 ('game', 'Women'): 1,
 ('loitering', 'important'): 0,
 ('dog', 'walk'): 1,
 ('playing', 'friendly'): 0,
 ('phone', 'people'): 1,
 ('pink', 'hospital